# **INF161 - Bike Traffic Prediction Project**
### *Ole Kristian Westby | owe009@uib.no | H23*

This project uses data from Statens vegvesen and Geofysisk institutt. The goal is to create a model that can predict the volume (how many) of bikers at a given time over Nygårdsbroen. I'll need to prepare, strip and sort the data so I'm left with the data I deem valuable to perform this task. That's what this Jupyter notebook is for. I'll also be explaining my steps throughout the book. At the end, we'll have some juicy, ready data that we'll use to insert into /ready_data/ ready for the model to work on.

## Let's start by importing some libraries.

In [5]:
import numpy as np
import os

## We'll handle the traffic data first.

In [6]:
dir_weather = "raw_data/weather_data/"
files = [f for f in os.listdir(dir_weather) if f.endswith('.csv')]
print(files)

